# Extraia o poder máximo dos seus modelos usando Stacking Ensembles - Live de Data Science #17

Tutorial for improve skills: 'Extraia o poder máximo dos seus modelos usando Stacking Ensembles - Live de Data Science #17' ( Mario Filho - Data Science) by Marcus Mariano

**For more information about Marcus Mariano: [Web site](https://marcusmariano.github.io/mmariano/)**  

**Extraia o poder máximo dos seus modelos usando Stacking Ensembles - Live de Data Science #17 [ Mario Filho - Data Science.](https://www.youtube.com/watch?v=TSoQGRhhHBE&t=27s)** 

In [1]:
import pandas as pd
import numpy as np

# Ensembles

- Join two or more Machine Learning model to a better solutions.
- Good for time series.
- Ensemble cluster can be used to unsupervised learning.
- With ensemble, it's can have earned 5% or 10% on final solution of your stronger Machine Learning model.

In [2]:
train = pd.read_csv("data/train.csv")
test = pd.read_csv("data/test.csv")

In [3]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [4]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass       418 non-null    int64  
 2   Name         418 non-null    object 
 3   Sex          418 non-null    object 
 4   Age          332 non-null    float64
 5   SibSp        418 non-null    int64  
 6   Parch        418 non-null    int64  
 7   Ticket       418 non-null    object 
 8   Fare         417 non-null    float64
 9   Cabin        91 non-null     object 
 10  Embarked     418 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 36.0+ KB


In [5]:
train['Sex_binario'] = train['Sex'].map({"male": 0, "female": 1})
test['Sex_binario'] = test['Sex'].map({"male": 0, "female": 1})

In [6]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Sex_binario
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,1
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,1
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,1
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,0


In [7]:
X = train.select_dtypes(include = np.number).drop(["PassengerId", 'Survived'], axis=1).fillna(0)
y = train['Survived']

In [8]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Pclass       891 non-null    int64  
 1   Age          891 non-null    float64
 2   SibSp        891 non-null    int64  
 3   Parch        891 non-null    int64  
 4   Fare         891 non-null    float64
 5   Sex_binario  891 non-null    int64  
dtypes: float64(2), int64(4)
memory usage: 41.9 KB


In [9]:
y[:10]

0    0
1    1
2    1
3    1
4    0
5    0
6    0
7    0
8    1
9    1
Name: Survived, dtype: int64

## Import model packages

In [10]:
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import KFold
from sklearn.metrics import log_loss, accuracy_score
from sklearn.pipeline import make_pipeline

In [11]:
np.zeros((X.shape[0], 4))

array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       ...,
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]])

In [12]:
np.zeros((X.shape[0], 4)).shape

(891, 4)

## fatores de diversidade

- Média e média ponderada (blending)
- Bagging e Boosting
- Média de Seeds
- Stacking: o queridinho do Kaggle

In [13]:
from tqdm.notebook import tqdm

In [20]:
# To resample
kf = KFold(n_splits=2, random_state=0, shuffle=True)

# Use a predictions of first model to training next models 
second_level = np.zeros((X.shape[0], 4))

for train, valid in tqdm(kf.split(X, y)):
    X_train, X_valid = X.iloc[train], X.iloc[valid]
    y_train, y_valid = y.iloc[train], y.iloc[valid]
    
    rf = RandomForestClassifier(n_estimators=100, n_jobs=6, random_state=10)
    rf.fit(X_train, y_train)
    pred_rf = rf.predict_proba(X_valid)[:,1]
    pred_rf_ = (pred_rf > 0.5).astype(int)
    
    print("RF Accuracy: {} - Log Loss: {}".format(accuracy_score(y_valid, pred_rf_), log_loss(y_valid, pred_rf)))
    
    et = ExtraTreesClassifier(n_estimators=100, n_jobs=6, random_state=10)
    et.fit(X_train, y_train)
    pred_et = et.predict_proba(X_valid)[:,1]
    pred_et_ = (pred_et > 0.5).astype(int)
    
    print("ET Accuracy: {} - Log Loss: {}".format(accuracy_score(y_valid, pred_et_), log_loss(y_valid, pred_et)))
    
    lr1 = make_pipeline(StandardScaler(), LogisticRegression())
    lr1.fit(X_train, y_train)
    pred_lr1 = lr1.predict_proba(X_valid)[:,1]
    pred_lr1_ = (pred_lr1 > 0.5).astype(int)
    
    print("LR StdScaler Accuracy: {} - Log Loss: {}".format(accuracy_score(y_valid, pred_lr1_), log_loss(y_valid, pred_lr1)))
    
    lr2 = make_pipeline(MinMaxScaler(), LogisticRegression())
    lr2.fit(X_train, y_train)
    pred_lr2 = lr2.predict_proba(X_valid)[:,1]
    pred_lr2_ = (pred_lr2 > 0.5).astype(int)
    
    print("LR MinMax Accuracy: {} - Log Loss: {}".format(accuracy_score(y_valid, pred_lr2_), log_loss(y_valid, pred_lr2)))
    
    second_level[valid, 0] = pred_rf
    second_level[valid, 1] = pred_et
    second_level[valid, 2] = pred_lr1
    second_level[valid, 3] = pred_lr2
    
    print()
    


RF Accuracy: 0.7869955156950673 - Log Loss: 0.8891877224352214
ET Accuracy: 0.7802690582959642 - Log Loss: 1.7277767849421113
LR StdScaler Accuracy: 0.7713004484304933 - Log Loss: 0.4548870233895985
LR MinMax Accuracy: 0.773542600896861 - Log Loss: 0.45730271249069515

RF Accuracy: 0.8292134831460675 - Log Loss: 0.6313360340802087
ET Accuracy: 0.8089887640449438 - Log Loss: 2.533916920090966
LR StdScaler Accuracy: 0.7955056179775281 - Log Loss: 0.4579224442106097
LR MinMax Accuracy: 0.802247191011236 - Log Loss: 0.4610789905245164




### Log Loss - 
- Random Forest is not calibrated by default

### Other forms of Stacking

- Simple Media
- Weighted Media

In [16]:
second_level

array([[0.15      , 0.19      , 0.0947167 , 0.11582552],
       [0.95      , 0.99      , 0.90010202, 0.88974478],
       [0.54      , 0.76      , 0.65737866, 0.63540321],
       ...,
       [0.74      , 0.76      , 0.61083421, 0.59159169],
       [0.84      , 0.92      , 0.45575654, 0.45086285],
       [0.35583333, 0.4       , 0.10479333, 0.11872796]])

In [18]:
# second_level.mean(axis=1)

In [ ]:
folds = [1, 2, 3]

primeiro nivel

ciclo 1 = [1,2] [3]
ciclo 2 = [1,3] [2]
ciclo 3 = [2,3] [1]

segundo nivel

ciclo 1 = [1,2] [3]
ciclo 2 = [1,3] [2]
ciclo 3 = [2,3] [1]






## Diversity Factors

- modelos - gbm + rede neural, knn 
- features
- exemplos 
- hiperparametros 

In [97]:
second_level

array([[0.15      , 0.19      , 0.0947167 , 0.11582552],
       [0.95      , 0.99      , 0.90010202, 0.88974478],
       [0.54      , 0.76      , 0.65737866, 0.63540321],
       ...,
       [0.74      , 0.76      , 0.61083421, 0.59159169],
       [0.84      , 0.92      , 0.45575654, 0.45086285],
       [0.35583333, 0.4       , 0.10479333, 0.11872796]])

In [21]:
# Does not recreate the kf to guarantee the same split
for train, valid in tqdm(kf.split(X,y)):
    
    X_train, X_valid = second_level[train], second_level[valid]
    y_train, y_valid = y.iloc[train], y.iloc[valid]
    
    lr_stack = LogisticRegression(C=1.)
    lr_stack.fit(X_train, y_train)
    pred_lr_stack = lr_stack.predict_proba(X_valid)[:,1]
    pred_lr_stack_ = (pred_lr_stack > 0.5).astype(int)
    
    print("Stack Accuracy: {}  Log loss: {}".format(accuracy_score(y_valid, pred_lr_stack_), 
                                                    log_loss(y_valid, pred_lr_stack)))
    print()

Stack Accuracy: 0.8004484304932735  Log loss: 0.4379152913562436

Stack Accuracy: 0.8157303370786517  Log loss: 0.42892608158477763




In [22]:
pd.DataFrame(np.corrcoef(second_level.T))

,0,1,2,3
0,1.000000,0.966815,0.808323,0.808835
1,0.966815,1.000000,0.756249,0.756569
2,0.808323,0.756249,1.000000,0.995999
3,0.808835,0.756569,0.995999,1.000000
